# Imports

In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pathlib
import io
import itertools
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp
import matplotlib.pyplot as plt

# Data

In [ ]:
# # Load CSV file
# csv = "butterflies/Training_set.csv"
# images = "butterflies/train"

# df = pd.read_csv(csv)

# # Generate full image paths
# df['image_path'] = df['filename'].apply(lambda x: os.path.join(images, x))

# # Convert labels to categorical if needed
# labels = df['label'].values
# image_paths = df['image_path'].values

# # Encode labels if they are categorical (string)
# label_encoder = LabelEncoder()
# labels = label_encoder.fit_transform(labels)
# df['label_encoded'] = labels

# number_of_labels = len(set(labels))
# number_of_labels

# train_paths, test_paths, train_labels, test_labels = train_test_split(
#     df['image_path'], df['label_encoded'], test_size=0.2, stratify=df['label_encoded'], random_state=42
# )
# val_paths, test_paths, val_labels, test_labels = train_test_split(
#     test_paths, test_labels, test_size=0.5, stratify=test_labels, random_state=42
# )

# IMG_X = 224
# IMG_Y = 224

# def load_image(image_path, label):
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (IMG_X, IMG_Y)) / 255.0  # Normalize
#     return image, label

# # Shuffle, batch, and prefetch
# BATCH_SIZE = 32

# # Convert lists into a TensorFlow dataset
# train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
# train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
# train_dataset = train_dataset.shuffle(len(train_paths)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
# val_dataset = val_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
# val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
# test_dataset = test_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
# test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Define paths
data_dir = pathlib.Path("flowers")  # Replace with your actual path

# Image settings
BATCH_SIZE = 32
IMG_X = 256
IMG_Y = 256
IMG_SIZE = (IMG_X, IMG_Y)  # Matches your image size

# Load training dataset (70%)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.3,  # 30% for val+test
    subset="training",
    seed=42
)

number_of_labels = len(train_dataset.class_names)  # Get number of unique labels
print(f"Number of classes: {number_of_labels}")
print(f"Class names: {train_dataset.class_names}")

val_test_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.3,  # 30% reserved for val+test
    subset="validation",  # This will contain val+test (30%)
    seed=42
)

# Step 2: Split val_test_dataset into Validation (15%) and Test (15%)
val_batches = int(0.5 * len(val_test_dataset))  # 50% of val+test for validation

val_dataset = val_test_dataset.take(val_batches)  # First half for validation
test_dataset = val_test_dataset.skip(val_batches)  # Second half for testing

# Prefetch data for better performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 13642 files belonging to 14 classes.
Using 9550 files for training.
Number of classes: 14
Class names: ['astilbe', 'bellflower', 'black_eyed_susan', 'calendula', 'california_poppy', 'carnation', 'common_daisy', 'coreopsis', 'dandelion', 'iris', 'rose', 'sunflower', 'tulip', 'water_lily']
Found 13642 files belonging to 14 classes.
Using 4092 files for validation.


# Model

In [38]:
# Defining some constants/hyperparameters
# BUFFER_SIZE = 70_000 # for reshuffling
NUM_EPOCHS = 20

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(kernel_size=3, filters=32, activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(number_of_labels)
# ])

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMG_X, IMG_Y, 3)),
    # First Conv Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Second Conv Block
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Third Conv Block
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Fourth Conv Block
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Flatten to convert feature maps to 1D
    tf.keras.layers.Flatten(),

    # Fully Connected Layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Reduces overfitting

    # Output Layer - 75 Classes
    tf.keras.layers.Dense(number_of_labels, activation='softmax')  # Softmax for multi-class classification
])

# Defining the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# Compiling the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [45]:
NUM_EPOCHS = 20  # Adjust based on performance
BATCH_SIZE = 32

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping]
)

Epoch 1/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 168s 561ms/step - accuracy: 0.1094 - loss: 16.7602 - val_accuracy: 0.2417 - val_loss: 2.1005
Epoch 2/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 168s 563ms/step - accuracy: 0.2715 - loss: 2.0656 - val_accuracy: 0.3892 - val_loss: 1.7979
Epoch 3/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 219s 734ms/step - accuracy: 0.4356 - loss: 1.6454 - val_accuracy: 0.4438 - val_loss: 1.6147
Epoch 4/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 166s 556ms/step - accuracy: 0.5597 - loss: 1.2903 - val_accuracy: 0.4795 - val_loss: 1.5567
Epoch 5/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 166s 556ms/step - accuracy: 0.6763 - loss: 0.9647 - val_accuracy: 0.4771 - val_loss: 1.7268
Epoch 6/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 167s 557ms/step - accuracy: 0.7647 - loss: 0.7005 - val_accuracy: 0.4990 - val_loss: 1.9750
Epoch 7/20
299/299 ━━━━━━━━━━━━━━━━━━━━ 167s 558ms/step - accuracy: 0.8361 - loss: 0.5015 - val_accuracy: 0.4971 - val_loss: 2.1906


In [46]:
test_loss, test_accuracy = model.evaluate(test_dataset)

64/64 ━━━━━━━━━━━━━━━━━━━━ 10s 150ms/step - accuracy: 0.4836 - loss: 1.5278


# Tuning

In [47]:
# # Defining the hyperparameters we would tune, and their values to be tested
# HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
# HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([32,64,96,128]))

# METRIC_ACCURACY = 'accuracy'

# # Logging setup info
# with tf.summary.create_file_writer(r'logs/model-1/hparam_tuning/').as_default():
#     hp.hparams_config(
#         hparams=[HP_FILTER_SIZE, HP_FILTER_NUM],
#         metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#     )

In [48]:
# NUM_EPOCHS = 20  # Adjust based on performance
# BATCH_SIZE = 32

In [49]:
# # Wrapping our model and training in a function
# def train_test_model(hparams, session_num):
    
#     # Outlining the model/architecture of our CNN
#     model = tf.keras.Sequential([
#         tf.keras.layers.Input(shape=(IMG_X, IMG_Y, 3)),
#         # First Conv Block
#         tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

#         # Second Conv Block
#         tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
#         tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

#         # Third Conv Block
#         tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
#         tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

#         # Fourth Conv Block
#         tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
#         tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

#         # Flatten to convert feature maps to 1D
#         tf.keras.layers.Flatten(),

#         # Fully Connected Layer
#         tf.keras.layers.Dense(512, activation='relu'),
#         tf.keras.layers.Dropout(0.3),  # Reduces overfitting

#         # Output Layer - 75 Classes
#         tf.keras.layers.Dense(75, activation='softmax')  # Softmax for multi-class classification
#     ])
    
#     # Defining the loss function
#     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

#     # Compiling the model
#     model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

#     # Defining the logging directory
#     log_dir = f"logs/model-1/fit/run-{session_num}"
    
#     # Define the Tensorboard and Confusion Matrix callbacks.
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

    
#     # Defining early stopping to prevent overfitting
#     early_stopping = tf.keras.callbacks.EarlyStopping(
#         monitor = 'val_loss',
#         mode = 'auto',
#         min_delta = 0,
#         patience = 3,
#         verbose = 0, 
#         restore_best_weights = True
#     )
    
#     # Training the model
#     model.fit(
#         train_dataset,
#         # images_train,
#         # labels_train,
#         epochs = NUM_EPOCHS,
#         batch_size = BATCH_SIZE,
#         callbacks = [tensorboard_callback, early_stopping],
#         validation_data = (val_dataset),
#         # validation_data = (images_val,labels_val),
#         verbose = 2
#     )
    
    
#     # Evaluating the model's performance on the validation set
#     _, accuracy = model.evaluate(val_dataset)
    
#     # Saving the current model for future reference
#     model.save(f"saved_models/model-1/run-{session_num}.keras")
    
#     return accuracy

In [50]:
# # Creating a function to log the resuls
# def run(log_dir, hparams, session_num):
    
#     with tf.summary.create_file_writer(log_dir).as_default():
#         hp.hparams(hparams)  # record the values used in this trial
#         accuracy = train_test_model(hparams, session_num)
#         tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [51]:
# session_num = 1

# for filter_size in HP_FILTER_SIZE.domain.values:
#     for filter_num in HP_FILTER_NUM.domain.values:

#         hparams = {
#             HP_FILTER_SIZE: filter_size,
#             HP_FILTER_NUM: filter_num
#         }

#         run_name = f"run-{session_num}"
#         print(f'--- Starting trial: {run_name}')
#         print({h.name: hparams[h] for h in hparams})
#         run('logs/model-1/hparam_tuning/' + run_name, hparams, session_num)

#         session_num += 1